# Snake game player versus player

This game was created inspired by the famous snake video game genre.

- The main objective is make the oponent player colide with your body or wait for him to colide with himself.

Rules are:

- Each game is divided in at least three rounds. When a player wins the round, he/she scores one point. A player needs three points to win the game.
- When a player hits a wall, its direction is changed to its left way.
- An apple appears randomly on screen and when a player reaches it, it is "eaten" and another apple reapears at a random spot again.
- Eating an apple lengthen the eater snake's body in a progressive manner: first apple gives 1 "squarebody", second apple gives 2, third gives 3, and so on.
- if players collide themselves head-to-head, the round ends in draw.

DISCLAIMER: This game was created for python learning purposes only and shall be free for anyone to base his or her project on my project. A little reference to me should be nice too.

## Library used

In [1]:
import random
import pygame
import time
from pygame.locals import *
from pygame import mixer

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


## Classes:

### A class for snake:

This is the main class in the game.
The snake is composed by a list of tuples where each tuple is a square with X and Y coordinates. The first tuple is the coordinates of the square, while the other tuples are bodyparts.

Method \__$init$__$(self,...)$
> - Here is stored the main attributes of the snake class. The main ones are the *self.body* which contains the coordinates of every square that a snake is composed, and the direction flags, which are booleans.
> - *self.key_locked*: its a boolean to help avoiding a player to change direction a second time until the head of the snake moves position, avoind a bug of own colision.

Method $.update(self)$
> - Movement occurs with "movement-flags", meaning that there are four direction flags (up, down, left, right), and therefore, four boolean variables. That means a movement south is when the "variable south" is true and the rest is false. Therefore, the way the game was shaped only allows pure vertical or pure horizontal movements.
> - The update of body parts positions occurs from tail to head. It needs to be this way, otherwise all squares would have the same position

Method $.display(self)$
> - A method that shows/blits the snake on screen.

In [2]:
class Snake(object):

    def __init__(self, start=[(200, 200), (210, 200), (220, 200)],
                 color=(120, 255, 0), name='Player_1', blink=False):
        """loading the starting snake, each tuple is the position of a segment of the snake."""
        self.body = start
        self.name = name
        self.score = [0]
        self.blink = blink
        self.key_locked = False

        # sizing the snake
        self.skin = pygame.Surface((10, 10))

        # colouring the snake:
        self.skin.fill(color)

        # starting directions
        self.fl_moving_right = False
        self.fl_moving_left = True
        self.fl_moving_down = False
        self.fl_moving_up = False

    def update(self):
        """This method updates the position of the snake. Each time the head moves one position forward, the 2nd square
        of the snake ocupies the previous position of the head, and the rest of the body continues this motion."""
        
        for i in range(len(self.body) - 1, 0, -1):
            self.body[i] = (self.body[i - 1][0], self.body[i - 1][1])

        # creating movements:
        if self.fl_moving_up == True:
            # moving up: Y lowers
            self.body[0] = (self.body[0][0], self.body[0][1] - 10) 
        if self.fl_moving_down == True:
            # moving down: Y rises
            self.body[0] = (self.body[0][0], self.body[0][1] + 10)  
        if self.fl_moving_left == True:
            # moving left: X lowers
            self.body[0] = (self.body[0][0] - 10, self.body[0][1])
        if self.fl_moving_right == True:
            # moving left: X lowers
            self.body[0] = (self.body[0][0] + 10, self.body[0][1])
        
        self.key_locked = False
            
    def display(self):
        """This method shows the snake on screen"""
        for pos in self.body:
            screen.blit(self.skin, pos)


### A a class for apple:

The class apple is very simple, its just need to be a red square that appears in a random position when the game is started or a previous apple is eaten.

In [3]:
class Apple(object):

    def __init__(self):
        self.position = (random_grid_pos())
        self.size = pygame.Surface((10, 10))  # sizing the apple
        self.size.fill((255, 0, 0))  # red color

### A class for the walls:

The walls are just an entity that aren't exactly needed, but were a way to practice classes and I would not need to modify or create a more complex background. They where also created in case the had any other future use in the game.

In [4]:
class Wall(object):
    """We are the watchers on the wall"""

    def __init__(self, wall_color=(0, 0, 0)):
        self.vertical_size = pygame.Surface((10, 600))
        self.horizontal_size = pygame.Surface((600, 10))
        self.horizontal_size.fill(wall_color)
        self.vertical_size.fill(wall_color)

    def display(self):
        # vertical walls (L,R)
        screen.blit(self.vertical_size, (0, 0))
        screen.blit(self.vertical_size, (590, 0))
        # horizontal walls (U,D)
        screen.blit(self.horizontal_size, (0, 0))
        screen.blit(self.horizontal_size, (0, 590))

## Functions:

### Generic/miscelaneous functions:

    - One for generating an random position for the apple.
    - One that checks collision between objects (returns a True when it happens).
    - One function that resets the movements of an object (snake).
    - One function that blinks a snake, when it hits itself or another player.
    - One function that prints on screen (not on console!) an text. A colision will update the specific msg to be printed after the snake battle ends.
    - One function that resets snakes parameters.
    - One function that checks scores of the snakes.


In [5]:
def random_grid_pos():
    '''This function standardizes the position of the apple
    so it can be generated in positions divisible by 10.
    This way the apple will aways fit properly with the snake'''
    
    x = random.randint(10, 580)
    y = random.randint(10, 580)
    return (x // 10 * 10, y // 10 * 10)

In [6]:
def collision(object1, object2):
    '''This functions return a boolean True when the first argument
    matches the position of the second argument'''
    
    return (object1[0] == object2[0]) and (object1[1] == object2[1])

In [7]:
def reset_movements(object1):
    '''This function reset all flag movements of an object/snake'''
    object1.fl_moving_up = False
    object1.fl_moving_down = False
    object1.fl_moving_left = False
    object1.fl_moving_right = False

In [8]:
def snake_blink(object1, object2, blink_amount=10, blink_color=(255, 255, 255), blink_freq=0.25):
    """This function blinks the snake when it hits itself or other snake"""

    white_squares = pygame.Surface((10, 10))
    white_squares.fill(blink_color)

    for i in range(blink_amount):

        if object1.blink:
            for pos in object1.body:
                screen.blit(white_squares, pos)

        if object2.blink:
            for pos in object2.body:
                screen.blit(white_squares, pos)

        pygame.display.flip()
        time.sleep(blink_freq)

        for pos in object1.body:
            screen.blit(object1.skin, pos)

        for pos in object2.body:
            screen.blit(object2.skin, pos)

        pygame.display.flip()
        time.sleep(blink_freq)

In [9]:
def print_on_screen(string='This is a test!',
                    string_color=(0, 255, 0), bkg_color=(0, 0, 128),
                    lfont='freesansbold.ttf', print_pos=(300, 300)):
    """Prints on screen a message."""
    
    # create a font object. 1st parameter is the font file, which is present in pygame. 2nd parameter is size of the font
    font = pygame.font.Font(lfont, 32)

    # create a text suface object, on which text is drawn on it.
    text = font.render(string, True, string_color, bkg_color)

    # create a rectangular object for the text surface object
    textRect = text.get_rect()

    # set the center of the rectangular object.
    textRect.center = print_pos
    screen.blit(text, textRect)

In [10]:
def reset_snake(object1,object2):
        """This function resets snakes movements and positions and other attributes"""

        # reset snakes direction:
        reset_movements(object1)
        reset_movements(object2)
        object1.fl_moving_left, object2.fl_moving_left = True, True

        # reset_snakes_position
        object1.body, object2.body = [(200, 200), (210, 200), (220, 200)], [(400, 400), (410, 400), (420, 400)]
        
        #reset other attributes
        object1.blink, object2.blink = False, False

In [11]:
def score_check(object1,score_needed=2):
    "Returns True if a player's score is enough to win the game. If so, turns global variable game_over True"
    global game_over
    
    if sum(object1.score) == score_needed:
        game_over = True
        return True
    else:
        return False

### Functions related to collisions:

- check_wall_colision(object)
- check_own_colision(object1, object2)
- check_head_colision(object1, object2)
- check_snake_colision(object1, object2)
- One function that checks all colisions



In [12]:
def check_wall_colision(object1):
    '''This function checks if the head of the snake colides with wall positions. If it does so,
    the course is diverted lef of the snake's current direction'''
    
    if object1.body[0][0] == screen.get_rect().right - 20 and object1.fl_moving_right == True:
        reset_movements(object1)
        object1.fl_moving_up = True
    
    if object1.body[0][0] == screen.get_rect().left + 10 and object1.fl_moving_left == True:
        reset_movements(object1)
        object1.fl_moving_down = True
    
    if object1.body[0][1] == screen.get_rect().bottom - 20 and object1.fl_moving_down == True:
        reset_movements(object1)
        object1.fl_moving_right = True
    
    if object1.body[0][1] == screen.get_rect().top + 10 and object1.fl_moving_up == True:
        reset_movements(object1)
        object1.fl_moving_left = True

In [13]:
def check_own_colision(object1, object2):
    """This function verifies if player collides with itself"""
    global round_over
    global round_end_msg

    if object1.body[0] in object1.body[1:]:
        round_end_msg = f'{object1.name} collided with himself!!'
        object1.blink = True
        round_over = True
        object2.score.append(1)

In [14]:
def check_head_colision(object1, object2):
    '''This function verifies if player collides with itself'''
    global round_over
    global round_end_msg

    if object1.body[0] == object2.body[0]:
        round_end_msg = f'Players collided head to head!!'
        round_over = True
        object1.blink = True
        object2.blink = True

In [15]:
def check_snake_colision(object1, object2):
    """This function verifies if player colides with another player
        it follows the way object1's head hits object2's body """
    global round_over
    global round_end_msg

    if object1.body[0] in object2.body[1:]:
        round_end_msg = f'{object2.name} scores!!!'
        object2.score.append(1)
        object1.blink = True
        round_over = True

In [16]:
def check_colisions(object1,object2):
    """This function checks all possible colisions"""
    
    check_head_colision(object1,object2)
    check_snake_colision(object1,object2)
    check_snake_colision(object2,object1)
    check_own_colision(object1,object2)
    check_own_colision(object2,object1)


### Functions related to checking actions:

- check_button_push(event, object, player_buttons=[K_w, K_s, K_a, K_d])
> - This function lets the player to press P for QUIT
> - It will also detect if a movement-related key is pressed (i.e.: up, down, left, right) and then change an object's (snake) movement. The key here is to not allow the player to make a movement to the oposite direction of the snake's way, so the snake won't colide with itself (or the head get inside the body).
> - The function also allows you to insert diferent keys (player_buttons) in order to change which keys are used to move the snake of player 1 or player 2
- check_click(event)
> - This function has two modes: one for when the game is not over and one for when the game is over.
> - this function is used to allow the game to procede in main menu or be restarted when it is over.


In [17]:
def check_button_push(event, object1, player_buttons=[K_w, K_s, K_a, K_d]):
    """This functions checks if a button was pushed and outputs an action"""
    global fight_started
    
    if event.type == KEYDOWN:

        if event.key == K_p:
            pygame.quit()

        if fight_started and not object1.key_locked:
            if object1.fl_moving_right == True or object1.fl_moving_left == True:
                if event.key == player_buttons[0]:
                    reset_movements(object1)
                    object1.fl_moving_up = True
                if event.key == player_buttons[1]:
                    reset_movements(object1)
                    object1.fl_moving_down = True

            if object1.fl_moving_down == True or object1.fl_moving_up == True:
                if event.key == player_buttons[2]:
                    reset_movements(object1)
                    object1.fl_moving_left = True
                if event.key == player_buttons[3]:
                    reset_movements(object1)
                    object1.fl_moving_right = True
            
            object1.key_locked = True

In [18]:
def check_click(event):
    '''This functions checks if a mouse clicks on an area related to a button on background'''
    
    global game_started
    global fight_started
    global round_over
    global restart

    if not game_over:
        if event.type == pygame.MOUSEBUTTONDOWN and not game_started:
            mouse_pos = pygame.mouse.get_pos()

            if mouse_pos[0] in range(200, 400, 1) and mouse_pos[1] in range(250, 350, 1):
                print('START button clicked!')
                game_started = True

            if mouse_pos[0] in range(200, 400, 1) and mouse_pos[1] in range(400, 500, 1):
                print('QUIT button clicked!')
                pygame.quit()

        if event.type == pygame.MOUSEBUTTONDOWN and game_started:
            mouse_pos = pygame.mouse.get_pos()

            if mouse_pos[0] in range(200, 400, 1) and mouse_pos[1] in range(400, 500, 1):
                print('GAME REALLY STARTED!')
                round_over = False
                fight_started = True

    else:
        if event.type == pygame.MOUSEBUTTONDOWN:
            mouse_pos = pygame.mouse.get_pos()

            if mouse_pos[0] in range(200, 400, 1) and mouse_pos[1] in range(420, 520, 1):
                print('BORA button clicked!')
                restart = True

### Functions related to event checking:

- check_events(object)
> - This function checks for events. First, checks events (pressed keys) for first player and then second player, or if the game is quit. Decisions are made after the player push a button.
> - Then, it checks colisions between the apple and the snake. Note that the apple_counter (it will be explained more about it later) rises. After eaten, a new square is appended to the snake body. The position of this new square starts outside of the visible display are, but as soon as a new
- check_main_menu_events(object1)

In [19]:
def check_events(object1):
    """This function check events within the game."""
    global apple_counter

    for event in pygame.event.get():
        check_button_push(event, snake)
        check_button_push(event, snake2, player_buttons=[K_UP, K_DOWN, K_LEFT, K_RIGHT])

        if event.type == QUIT:
            pygame.quit()

    # Seting colision apple + snake
    if collision(object1.body[0], apple.position):
        apple.position = random_grid_pos()
        apple_counter.append(0)

        for i in apple_counter:
            object1.body.append((-10, -10))

    check_wall_colision(object1)

In [20]:
def check_main_menu_events(object1):
    """Checks all main menu events and detects mouse clicks or button pushs"""
    
    for event in pygame.event.get():
        check_button_push(event, object1)
        check_click(event)
        print(event)

        if event.type == QUIT:
            pygame.quit()

## The game algorithm

### Pre-declared variables:

In [21]:
round_end_msg = 'Round end!'
apple_counter = []
scores = [[], []]
intro_sound = 'game_intro.wav'
menu_player_sound = 'player_menu.mp3'
game_back_sound = 'game_fight.mp3'
winner = 'winner.mp3' 
death = 'death.wav'
hit = '2AH.wav'

# assigning values to X and Y variable: display dimensions
X = 600
Y = 600

### Initializing pygame: loading screen display and sound effects

Initializing pygame and display

In [22]:
pygame.init()
pygame.mixer.pre_init(44100, 16, 2, 4096)  # frequency, size, channels, buffersize

# Loading pygame and setting a display size with window name.
screen = pygame.display.set_mode((X, Y))
pygame.display.set_caption('Snek PvP Game')
clock = pygame.time.Clock()

Sounds and background images:

In [23]:
background = pygame.image.load('menu.png')
background2 = pygame.image.load('menu_player.png')
background_end1 = pygame.image.load('end_bkg1.png')
background_end2 = pygame.image.load('end_bkg2.png')
death_sound = pygame.mixer.Sound(death)
hit_sound = pygame.mixer.Sound(hit)

### Main loop

The game operates with an while loop with a main while inside.
Inside this main loop (*while not game_over*), there are four sub loops:

- When the main loop starts, All boolean variables that rule the sub-loops are restarted.

- First "sub" loop is the *main menu*, with an background made in paint.
> *while not game_started:*
> - Here there are two options: if the player clicks within the "Start" button area, *game_started* changes to *True*

- Second "sub" loop is the *player menu*, with another background made in paint.
> *while round_over:*
> - Here there is only one option: the button "BORA" that starts the round. If the player clicks within the "BORA" button area, *round_over* changes to *False*

- Third "sub" loop is the *snake battle*, with background and entities generated by the pygame functions.
> *while not round_over:*
> - Here, if any colision happens, the score is changed and the *round_over* changes back to *True*.
> - If any player score two points total, the variable game_over will be True, and the fourth sub loop starts.

- Fourth "sub" loop is the *end game menu*, with backgroundalso made in paint.
> *while not restart:*
> - Here, there is only one option is to click within on the virtual button "Back to main menu". if clicked, restart = True, and the players are sent back to main menu.

In [24]:
while True:
    game_over = False
    round_over = True
    game_started = False
    fight_started = False
    restart = False
    apple = Apple()
    wall = Wall()
    bg_color = (230, 230, 230)

    # Background Sound
    pygame.mixer.init()
    mixer.music.load(intro_sound)
    mixer.music.play(-1)

    # generating snakes
    snake = Snake(color=(0, 0, 255), )
    snake2 = Snake(start=[(400, 400), (410, 400), (420, 400)], color=(255, 255, 0), name='Player_2')

    #The main game loop
    while not game_over:

        while not game_started:
            #First Menu
            screen.blit(background, [0, 0])
            check_main_menu_events(object1=[])
            pygame.display.flip()

        mixer.music.load(menu_player_sound)
        mixer.music.play(-1)

        while round_over:
            #Player Menu
            screen.blit(background2, [0, 0])
            check_main_menu_events(object1=[])
            print_on_screen(f'Wins: {sum(snake.score)}' + ' ' * 30 + f'Wins: {sum(snake2.score)}',
                            print_pos=(300, 50), string_color=(255, 255, 255), bkg_color=(0, 0, 0))
            pygame.display.flip()

        # reset snakes direction, position and apple_counter:
        reset_snake(snake, snake2)
        apple_counter = []

        mixer.music.load(game_back_sound)
        mixer.music.play(-1)

        while not round_over:
            clock.tick(20)
            check_events(snake)
            check_events(snake2)
            snake.update()
            snake2.update()

            # updating screen
            screen.fill((140, 140, 50))
            screen.blit(apple.size, apple.position)
            wall.display()
            snake.display()
            snake2.display()
            pygame.display.flip()
            
            #check all colisions
            check_colisions(snake, snake2)

        print_on_screen(round_end_msg, bkg_color=(140, 140, 50), string_color=(255, 0, 0))
        death_sound.play()
        hit_sound.play(5)
        snake_blink(snake, snake2)
        round_over = True
        fight_started = False
        time.sleep(1)
        
        if score_check(snake):
             bkg_end = background_end1

        if score_check(snake2):
             bkg_end = background_end2

    mixer.music.load(winner)
    mixer.music.play(-1)

    while not restart:
        fight_started = False
        game_started = False
        screen.blit(bkg_end, [0, 0])
        check_main_menu_events(object1=[])
        pygame.display.flip()

<Event(17-VideoExpose {})>
<Event(16-VideoResize {'size': (600, 600), 'w': 600, 'h': 600})>
<Event(1-ActiveEvent {'gain': 0, 'state': 1})>
<Event(4-MouseMotion {'pos': (599, 127), 'rel': (600, 128), 'buttons': (0, 0, 0), 'window': None})>
<Event(1-ActiveEvent {'gain': 1, 'state': 1})>
<Event(4-MouseMotion {'pos': (28, 359), 'rel': (-571, 232), 'buttons': (0, 0, 0), 'window': None})>
<Event(4-MouseMotion {'pos': (100, 373), 'rel': (72, 14), 'buttons': (0, 0, 0), 'window': None})>
<Event(4-MouseMotion {'pos': (174, 389), 'rel': (74, 16), 'buttons': (0, 0, 0), 'window': None})>
<Event(4-MouseMotion {'pos': (209, 398), 'rel': (35, 9), 'buttons': (0, 0, 0), 'window': None})>
<Event(4-MouseMotion {'pos': (244, 406), 'rel': (35, 8), 'buttons': (0, 0, 0), 'window': None})>
<Event(4-MouseMotion {'pos': (302, 425), 'rel': (58, 19), 'buttons': (0, 0, 0), 'window': None})>
<Event(4-MouseMotion {'pos': (321, 432), 'rel': (19, 7), 'buttons': (0, 0, 0), 'window': None})>
<Event(4-MouseMotion {'pos': 

<Event(4-MouseMotion {'pos': (179, 327), 'rel': (3, -2), 'buttons': (0, 0, 0), 'window': None})>
<Event(4-MouseMotion {'pos': (203, 313), 'rel': (24, -14), 'buttons': (0, 0, 0), 'window': None})>
<Event(4-MouseMotion {'pos': (268, 306), 'rel': (65, -7), 'buttons': (0, 0, 0), 'window': None})>
<Event(4-MouseMotion {'pos': (312, 308), 'rel': (44, 2), 'buttons': (0, 0, 0), 'window': None})>
<Event(4-MouseMotion {'pos': (318, 312), 'rel': (6, 4), 'buttons': (0, 0, 0), 'window': None})>
<Event(4-MouseMotion {'pos': (319, 313), 'rel': (1, 1), 'buttons': (0, 0, 0), 'window': None})>
START button clicked!
<Event(5-MouseButtonDown {'pos': (319, 313), 'button': 1, 'window': None})>
<Event(6-MouseButtonUp {'pos': (319, 313), 'button': 1, 'window': None})>
<Event(2-KeyDown {'unicode': '', 'key': 273, 'mod': 0, 'scancode': 72, 'window': None})>
<Event(3-KeyUp {'key': 273, 'mod': 0, 'scancode': 72, 'window': None})>
<Event(4-MouseMotion {'pos': (322, 313), 'rel': (3, 0), 'buttons': (0, 0, 0), 'windo

<Event(4-MouseMotion {'pos': (443, 451), 'rel': (0, -2), 'buttons': (0, 0, 0), 'window': None})>
<Event(4-MouseMotion {'pos': (450, 438), 'rel': (7, -13), 'buttons': (0, 0, 0), 'window': None})>
<Event(4-MouseMotion {'pos': (458, 423), 'rel': (8, -15), 'buttons': (0, 0, 0), 'window': None})>
<Event(4-MouseMotion {'pos': (460, 421), 'rel': (2, -2), 'buttons': (0, 0, 0), 'window': None})>
<Event(4-MouseMotion {'pos': (461, 423), 'rel': (1, 2), 'buttons': (0, 0, 0), 'window': None})>
<Event(4-MouseMotion {'pos': (462, 424), 'rel': (1, 1), 'buttons': (0, 0, 0), 'window': None})>
<Event(4-MouseMotion {'pos': (466, 428), 'rel': (4, 4), 'buttons': (0, 0, 0), 'window': None})>
<Event(4-MouseMotion {'pos': (470, 430), 'rel': (4, 2), 'buttons': (0, 0, 0), 'window': None})>
<Event(4-MouseMotion {'pos': (472, 432), 'rel': (2, 2), 'buttons': (0, 0, 0), 'window': None})>
<Event(4-MouseMotion {'pos': (474, 434), 'rel': (2, 2), 'buttons': (0, 0, 0), 'window': None})>
<Event(4-MouseMotion {'pos': (480,

<Event(4-MouseMotion {'pos': (494, 536), 'rel': (1, 1), 'buttons': (0, 0, 0), 'window': None})>
<Event(4-MouseMotion {'pos': (496, 538), 'rel': (2, 2), 'buttons': (0, 0, 0), 'window': None})>
<Event(4-MouseMotion {'pos': (497, 538), 'rel': (1, 0), 'buttons': (0, 0, 0), 'window': None})>
<Event(4-MouseMotion {'pos': (499, 540), 'rel': (2, 2), 'buttons': (0, 0, 0), 'window': None})>
<Event(4-MouseMotion {'pos': (501, 542), 'rel': (2, 2), 'buttons': (0, 0, 0), 'window': None})>
<Event(4-MouseMotion {'pos': (503, 546), 'rel': (2, 4), 'buttons': (0, 0, 0), 'window': None})>
<Event(4-MouseMotion {'pos': (505, 546), 'rel': (2, 0), 'buttons': (0, 0, 0), 'window': None})>
<Event(4-MouseMotion {'pos': (507, 547), 'rel': (2, 1), 'buttons': (0, 0, 0), 'window': None})>
<Event(4-MouseMotion {'pos': (508, 547), 'rel': (1, 0), 'buttons': (0, 0, 0), 'window': None})>
<Event(4-MouseMotion {'pos': (509, 547), 'rel': (1, 0), 'buttons': (0, 0, 0), 'window': None})>
<Event(4-MouseMotion {'pos': (511, 546),

error: video system not initialized

## References and used media:

Soundtrack was obtained from youtube videos:

- 'game_intro.wav' - https://www.youtube.com/watch?v=vMqUEFIb-EI&list=PL709C66DD40795D4F
- 'player_menu.mp3' - https://www.youtube.com/watch?v=Kmnmaui9s_I
- 'game_fight.mp3' - https://www.youtube.com/watch?v=TBR2x1v-5s4
- 'winner.mp3' - https://www.youtube.com/watch?v=aSQ00SAPs1g
- 'death.wav' - https://www.youtube.com/watch?v=3FmN46XQius
- '2AH.wav' - https://www.youtube.com/watch?v=axV4iFiuDmE